Business case: In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

In [1]:
!pip install imblearn

In [2]:
import imblearn
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
password = getpass.getpass()

········


1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.
2. Read the data into a Pandas dataframe.

In [3]:
#first query to retrieve rentals counted per month for each film_id:
connection_string = 'mysql+pymysql://root:' + 'Strike.1612' + '@localhost/sakila'
engine = create_engine(connection_string)

query = '''
        with cte_monthly_data as (
            select rental_id, inventory_id, convert(rental_date, date) as Rental_date, 
                date_format(convert(rental_date, date), '%%m') as Rental_month, 
                date_format(convert(rental_date, date), '%%Y') as Rental_year
            from sakila.rental
            )
        select f.film_id, count(md.rental_id) as rentals_counted, md.rental_month, md.rental_year, f.rental_duration, f.rating from sakila.film f
        join sakila.film_category fc using (film_id)
        join sakila.inventory i using (film_id)
        join cte_monthly_data md using (inventory_id)
        group by f.film_id, md.rental_month, md.rental_year;
        '''

data = pd.read_sql_query(query, engine)
data

,film_id,rentals_counted,Rental_month,Rental_year,rental_duration,rating
0,19,7,08,2005,6,PG
1,19,4,06,2005,6,PG
2,19,8,07,2005,6,PG
3,19,1,05,2005,6,PG
4,19,1,02,2006,6,PG
...,...,...,...,...,...,...
3666,988,2,06,2005,7,R
3667,989,5,06,2005,4,R
3668,989,10,07,2005,4,R
3669,989,8,08,2005,4,R


In [4]:
#removing the data for 2023 (not in the original dataset, added during the lab)
data = data[data.Rental_year != "2023"]
data

,film_id,rentals_counted,Rental_month,Rental_year,rental_duration,rating
0,19,7,08,2005,6,PG
1,19,4,06,2005,6,PG
2,19,8,07,2005,6,PG
3,19,1,05,2005,6,PG
4,19,1,02,2006,6,PG
...,...,...,...,...,...,...
3666,988,2,06,2005,7,R
3667,989,5,06,2005,4,R
3668,989,10,07,2005,4,R
3669,989,8,08,2005,4,R


In [5]:
# My original dataset from Sakila doesn't have rows for movies that weren't rented in a given month (therefore all the movies have status B) - I will create another dataframe based on the previous df, using pivot table. 
counted_data = pd.pivot_table(data, columns=['Rental_month', 'Rental_year'], index=['film_id'], values=['rentals_counted'], aggfunc=np.sum)
counted_data

rentals_counted                      
Rental_month              02   05   06    07    08
Rental_year             2006 2005 2005  2005  2005
film_id                                           
1                        NaN  2.0  3.0   9.0   9.0
2                        1.0  NaN  NaN   2.0   4.0
3                        NaN  1.0  1.0   4.0   6.0
4                        1.0  2.0  2.0  12.0   6.0
5                        1.0  1.0  2.0   5.0   3.0
...                      ...  ...  ...   ...   ...
996                      NaN  NaN  1.0   3.0   3.0
997                      NaN  NaN  1.0   2.0   3.0
998                      1.0  1.0  2.0   3.0   2.0
999                      NaN  1.0  2.0   7.0   7.0
1000                     NaN  3.0  5.0  12.0  11.0

[958 rows x 5 columns]

In [6]:
#pivot table returned NaN values for the months in which a given movie wasn't rented - I will replace them with 0:
counted_data = counted_data.fillna(0)
counted_data

rentals_counted                      
Rental_month              02   05   06    07    08
Rental_year             2006 2005 2005  2005  2005
film_id                                           
1                        0.0  2.0  3.0   9.0   9.0
2                        1.0  0.0  0.0   2.0   4.0
3                        0.0  1.0  1.0   4.0   6.0
4                        1.0  2.0  2.0  12.0   6.0
5                        1.0  1.0  2.0   5.0   3.0
...                      ...  ...  ...   ...   ...
996                      0.0  0.0  1.0   3.0   3.0
997                      0.0  0.0  1.0   2.0   3.0
998                      1.0  1.0  2.0   3.0   2.0
999                      0.0  1.0  2.0   7.0   7.0
1000                     0.0  3.0  5.0  12.0  11.0

[958 rows x 5 columns]

In [7]:
# to simplify the dataframe, I will join the headers to keep them in one line: 
counted_data.columns = counted_data.columns.map('_'.join)

In [8]:
# Since my model will be predicting next month (03/2006) based on the last month (02/2006), I will keep only the column for Feb'06
counted_data = counted_data.drop(columns = ['rentals_counted_05_2005', 'rentals_counted_06_2005', 'rentals_counted_07_2005', 'rentals_counted_08_2005'], axis=1)

In [9]:
counted_data.shape

(958, 1)

In [10]:
# I will upload another table with more information about film features
connection_string = 'mysql+pymysql://root:' + 'Strike.1612' + '@localhost/sakila'
engine = create_engine(connection_string)

query = '''
        select film_id, rental_duration, rental_rate, rating, special_features
        from sakila.film;
        '''

film_data = pd.read_sql_query(query, engine)
film_data

,film_id,rental_duration,rental_rate,rating,special_features
0,1,6,0.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,3,4.99,G,"Trailers,Deleted Scenes"
2,3,7,2.99,NC-17,"Trailers,Deleted Scenes"
3,4,5,2.99,G,"Commentaries,Behind the Scenes"
4,5,6,2.99,G,Deleted Scenes
...,...,...,...,...,...
995,996,6,0.99,G,"Trailers,Behind the Scenes"
996,997,4,0.99,NC-17,"Trailers,Behind the Scenes"
997,998,6,0.99,NC-17,Deleted Scenes
998,999,5,2.99,R,"Trailers,Deleted Scenes"


In [11]:
# I will merge two dataframes together: counted_data and film_data: 
films_data = pd.merge(counted_data, film_data, how='left', on=['film_id'])
films_data.shape

(958, 6)

In [12]:
films_data.head()

,film_id,rentals_counted_02_2006,rental_duration,rental_rate,rating,special_features
0,1,0.0,6,0.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,1.0,3,4.99,G,"Trailers,Deleted Scenes"
2,3,0.0,7,2.99,NC-17,"Trailers,Deleted Scenes"
3,4,1.0,5,2.99,G,"Commentaries,Behind the Scenes"
4,5,1.0,6,2.99,G,Deleted Scenes


3. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [13]:
films_data.shape

(958, 6)

In [14]:
films_data.dtypes 

film_id                      int64
rentals_counted_02_2006    float64
rental_duration              int64
rental_rate                float64
rating                      object
special_features            object
dtype: object

In [15]:
#changing float variables into integers
films_data.rentals_counted_02_2006 = films_data.rentals_counted_02_2006.astype(int)
films_data.rental_rate = films_data.rental_rate.round().astype(int)

In [16]:
films_data.dtypes 

film_id                     int64
rentals_counted_02_2006     int32
rental_duration             int64
rental_rate                 int32
rating                     object
special_features           object
dtype: object

In [17]:
films_data.isna().sum()

film_id                    0
rentals_counted_02_2006    0
rental_duration            0
rental_rate                0
rating                     0
special_features           0
dtype: int64

In [18]:
films_data['rating'].value_counts()

PG-13    213
NC-17    202
R        189
PG       183
G        171
Name: rating, dtype: int64

In [19]:
films_data['special_features'].value_counts() #this column will require encoding

Trailers,Commentaries,Behind the Scenes                   76
Trailers                                                  71
Commentaries,Behind the Scenes                            70
Deleted Scenes,Behind the Scenes                          68
Behind the Scenes                                         68
Trailers,Commentaries                                     67
Trailers,Behind the Scenes                                67
Trailers,Deleted Scenes                                   65
Commentaries,Deleted Scenes,Behind the Scenes             65
Commentaries,Deleted Scenes                               62
Commentaries                                              61
Trailers,Commentaries,Deleted Scenes                      60
Deleted Scenes                                            57
Trailers,Commentaries,Deleted Scenes,Behind the Scenes    55
Trailers,Deleted Scenes,Behind the Scenes                 46
Name: special_features, dtype: int64

In [20]:
films_data1 = films_data.copy()

In [21]:
import re 
def trailers_values(x):
    x =str(x).lower()
    if re.search(r'\btrailers\b', x):
        return "1"
    else:
        return "0"

In [22]:
films_data1["trailers"] = films_data1["special_features"].apply(trailers_values)

In [23]:
def deleted_scenes_values(x):
    x =str(x).lower()
    if re.search(r'\bdeleted\b', x):
        return "1"
    else:
        return "0"

In [24]:
films_data1["deleted_scenes"] = films_data1["special_features"].apply(deleted_scenes_values)

In [25]:
def behind_the_scenes_values(x):
    x =str(x).lower()
    if re.search(r'\bbehind\b', x):
        return "1"
    else:
        return "0"

In [26]:
films_data1["behind_the_scenes"] = films_data1["special_features"].apply(behind_the_scenes_values)

In [27]:
def commentaries_values(x):
    x =str(x).lower()
    if re.search(r'\bcommentaries\b', x):
        return "1"
    else:
        return "0"

In [28]:
films_data1["commentaries"] = films_data1["special_features"].apply(commentaries_values)

In [29]:
films_data1

,film_id,rentals_counted_02_2006,rental_duration,rental_rate,rating,special_features,trailers,deleted_scenes,behind_the_scenes,commentaries
0,1,0,6,1,PG,"Deleted Scenes,Behind the Scenes",0,1,1,0
1,2,1,3,5,G,"Trailers,Deleted Scenes",1,1,0,0
2,3,0,7,3,NC-17,"Trailers,Deleted Scenes",1,1,0,0
3,4,1,5,3,G,"Commentaries,Behind the Scenes",0,0,1,1
4,5,1,6,3,G,Deleted Scenes,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
953,996,0,6,1,G,"Trailers,Behind the Scenes",1,0,1,0
954,997,0,4,1,NC-17,"Trailers,Behind the Scenes",1,0,1,0
955,998,1,6,1,NC-17,Deleted Scenes,0,1,0,0
956,999,0,5,3,R,"Trailers,Deleted Scenes",1,1,0,0


In [30]:
films_data2 = films_data1.copy()

In [31]:
films_data2 = films_data2.drop(columns = ['special_features'], axis=1) #now I can drop special features columns

In [32]:
films_data2

,film_id,rentals_counted_02_2006,rental_duration,rental_rate,rating,trailers,deleted_scenes,behind_the_scenes,commentaries
0,1,0,6,1,PG,0,1,1,0
1,2,1,3,5,G,1,1,0,0
2,3,0,7,3,NC-17,1,1,0,0
3,4,1,5,3,G,0,0,1,1
4,5,1,6,3,G,0,1,0,0
...,...,...,...,...,...,...,...,...,...
953,996,0,6,1,G,1,0,1,0
954,997,0,4,1,NC-17,1,0,1,0
955,998,1,6,1,NC-17,0,1,0,0
956,999,0,5,3,R,1,1,0,0


In [33]:
films_data2.dtypes

film_id                     int64
rentals_counted_02_2006     int32
rental_duration             int64
rental_rate                 int32
rating                     object
trailers                   object
deleted_scenes             object
behind_the_scenes          object
commentaries               object
dtype: object

In [34]:
films_data2['trailers'] = films_data2['trailers'].astype(int)
films_data2['deleted_scenes'] = films_data2['deleted_scenes'].astype(int)
films_data2['behind_the_scenes'] = films_data2['behind_the_scenes'].astype(int)
films_data2['commentaries'] = films_data2['commentaries'].astype(int)

In [35]:
films_data2.dtypes

film_id                     int64
rentals_counted_02_2006     int32
rental_duration             int64
rental_rate                 int32
rating                     object
trailers                    int32
deleted_scenes              int32
behind_the_scenes           int32
commentaries                int32
dtype: object

In [36]:
# I will encode rating later. 

4. Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.

In [37]:
# I will use the existing dataframe and create a new column 'status' which will define whether the movie was rented or not (based on the content of 'rentals_counted_02_2006'):
def status (x):
    if x['rentals_counted_02_2006'] == 0.0: 
        return "B" #status B = movie hasn't been rented out last month
    if x['rentals_counted_02_2006'] > 0.0:
        return "A" #status A = movie has been rented out last month

In [38]:
films_data2['status'] = films_data2.apply(lambda row: status(row), axis=1)

In [39]:
films_data3 = films_data2.drop(columns = ['rentals_counted_02_2006'], axis=1) #dropping this column to avoid data leakage 

In [40]:
films_data3

,film_id,rental_duration,rental_rate,rating,trailers,deleted_scenes,behind_the_scenes,commentaries,status
0,1,6,1,PG,0,1,1,0,B
1,2,3,5,G,1,1,0,0,A
2,3,7,3,NC-17,1,1,0,0,B
3,4,5,3,G,0,0,1,1,A
4,5,6,3,G,0,1,0,0,A
...,...,...,...,...,...,...,...,...,...
953,996,6,1,G,1,0,1,0,B
954,997,4,1,NC-17,1,0,1,0,B
955,998,6,1,NC-17,0,1,0,0,A
956,999,5,3,R,1,1,0,0,B


5. Create a logistic regression model to predict this variable from the cleaned data.

In [41]:
y = films_data3['status']
X = films_data3.drop('status', axis=1)

In [42]:
X = pd.get_dummies(X,columns=['rating']) #encoding categorical
X

,film_id,rental_duration,rental_rate,trailers,deleted_scenes,behind_the_scenes,commentaries,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,1,6,1,0,1,1,0,0,0,1,0,0
1,2,3,5,1,1,0,0,1,0,0,0,0
2,3,7,3,1,1,0,0,0,1,0,0,0
3,4,5,3,0,0,1,1,1,0,0,0,0
4,5,6,3,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
953,996,6,1,1,0,1,0,1,0,0,0,0
954,997,4,1,1,0,1,0,0,1,0,0,0
955,998,6,1,0,1,0,0,0,1,0,0,0
956,999,5,3,1,1,0,0,0,0,0,0,1


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #splitting the data

In [44]:
#scaling the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [45]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [46]:
classification = LogisticRegression(random_state=42) #training the model

classification.fit(X_train_scaled, y_train)

LogisticRegression(random_state=42)

In [47]:
classification.score(X_test_scaled, y_test) #getting the accuracy

0.8645833333333334

In [48]:
predictions = classification.predict(X_test_scaled) #getting predictions
predictions

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B

In [49]:
y_test.value_counts() #checking if the dataset is balanced (it's not as we have way more 'B')

B    166
A     26
Name: status, dtype: int64

In [50]:
confusion_matrix(y_test, predictions)

array([[  0,  26],
       [  0, 166]], dtype=int64)

6. Evaluate the results.

With the accuracy close to 1 (0.86) the model is working well with 86% of accuracy on the given dataset. But since the dataset is very imbalanced towards B (166 vs 26). The result of that is visible in the confusion matrix - the model didn't predict any results for status A. Therefore in order to deal with the class imbalance further techniques should be applied (oversampling, undersampling) to improve the model. 